In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'E:\\Document\\HK2-2022-2023\\DIPR\\Project\\research\\GrapeDiseaseDataset\\OriginalData\\train'
test_dir = 'E:\\Document\\HK2-2022-2023\\DIPR\\Project\\research\\GrapeDiseaseDataset\\OriginalData\\test'



In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

Found 7222 images belonging to 4 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

Found 1805 images belonging to 4 classes.


# First test model

In [ ]:
model = None
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


# Second test model with dropout

In [ ]:
model = None
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


# ResNet 50

In [ ]:
# Define the input shape
input_shape = (256, 256, 3)

# Define the number of classes
num_classes = 4

# Define the model architecture
inputs = tf.keras.Input(shape=input_shape)

# Stage 1
x = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2), padding='same')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

# Stage 2
shortcut = x
x = tf.keras.layers.Conv2D(64, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

# Stage 3
shortcut = x
x = tf.keras.layers.Conv2D(128, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(128, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(512, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(512, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Stage 4
shortcut = x
x = tf.keras.layers.Conv2D(256, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(1024, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(1024, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(1024, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Stage 5
shortcut = x
x = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(512, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(2048, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(2048, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(2048, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Final layers
x = tf.keras.layers.AveragePooling2D((7, 7))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(4, activation='softmax')(x)

#Create Model
model = None
model = tf.keras.models.Model(inputs=inputs, outputs=x)
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


# With dropout

In [4]:
# Define the input shape
input_shape = (256, 256, 3)

# Define the number of classes
num_classes = 4

# Define the model architecture
inputs = tf.keras.Input(shape=input_shape)

# Stage 1
x = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2), padding='same')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

# Stage 2
shortcut = x
x = tf.keras.layers.Conv2D(64, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

# Stage 3
shortcut = x
x = tf.keras.layers.Conv2D(128, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(128, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(512, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(512, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Stage 4
shortcut = x
x = tf.keras.layers.Conv2D(256, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(1024, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(1024, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(1024, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Stage 5
shortcut = x
x = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(512, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(2048, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(2048, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(2048, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

# Final layers
x = tf.keras.layers.AveragePooling2D((7, 7))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(4, activation='softmax')(x)

#Create Model
model = None
model = tf.keras.models.Model(inputs=inputs, outputs=x)
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [5]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator
)


Epoch 1/50


c:\Users\TVD\anaconda3\envs\tf\lib\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


226/226 [==============================] - 240s 973ms/step - loss: 0.5346 - accuracy: 0.8228 - val_loss: 1.3710 - val_accuracy: 0.4975
Epoch 2/50
226/226 [==============================] - 87s 384ms/step - loss: 0.2709 - accuracy: 0.9050 - val_loss: 10.5966 - val_accuracy: 0.4460
Epoch 3/50
226/226 [==============================] - 86s 378ms/step - loss: 0.2305 - accuracy: 0.9223 - val_loss: 2.5949 - val_accuracy: 0.6576
Epoch 4/50
226/226 [==============================] - 85s 377ms/step - loss: 0.1532 - accuracy: 0.9493 - val_loss: 4.3894 - val_accuracy: 0.6271
Epoch 5/50
226/226 [==============================] - 93s 408ms/step - loss: 0.1390 - accuracy: 0.9556 - val_loss: 2.0006 - val_accuracy: 0.6781
Epoch 6/50
226/226 [==============================] - 92s 404ms/step - loss: 0.1176 - accuracy: 0.9601 - val_loss: 19.3437 - val_accuracy: 0.3330
Epoch 7/50
226/226 [==============================] - 112s 497ms/step - loss: 0.1010 - accuracy: 0.9651 - val_loss: 29.8532 - val_accuracy

In [6]:
model.evaluate(test_generator)

57/57 [==============================] - 5s 79ms/step - loss: 1.1819 - accuracy: 0.8122


[1.1819442510604858, 0.8121883869171143]

# Save first model

In [ ]:
# Save the trained model as a .h5 file
model.save('my_trained_model.h5')

# Save second model

In [ ]:
# Save the trained model as a .h5 file
model.save('my_trained_model2.h5')

# Save ResNet

In [ ]:
# Save the trained model as a .h5 file
model.save('my_trained_model3.h5')

# Save ResNet with droput

In [7]:
# Save the trained model as a .h5 file
model.save('my_trained_model4.h5')

In [8]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import ImageTk, Image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('my_trained_model4.h5')
img_height = img_width = 256
# Define the class names
class_names = ['Black Rot', 'ESCA', 'Healthy', 'Leaf Blight', ...]

# Create a function to preprocess the image
def preprocess_image(image):
    # Resize the image to match the input size of the model
    image = image.resize((img_height, img_width))
    # Convert the image to a numpy array
    image_array = np.array(image)
    # Normalize the pixel values to be between 0 and 1
    image_array = image_array / 255.0
    # Expand the dimensions of the image to match the input shape of the model
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

# Create a function to classify the image
def classify_image(image_path):
    # Load the image
    image = Image.open(image_path)
    # Preprocess the image
    image_array = preprocess_image(image)
    # Use the model to make predictions
    predictions = model.predict(image_array)
    # Get the class with the highest probability
    predicted_class = np.argmax(predictions[0])
    # Get the class name
    class_name = class_names[predicted_class]
    # Get the probability of the predicted class
    class_prob = np.max(predictions[0])
    # Convert the probability to a percentage
    class_percent = round(class_prob * 100, 2)
    # Show a message box with the predicted class and percentage
    messagebox.showinfo('Classification Result', f'The image is classified as {class_name} with {class_percent}% probability.')

# Create a function to handle the upload button click
def handle_upload():
    # Open a file dialog to select the image file
    file_path = filedialog.askopenfilename()
    # Check if a file was selected
    if file_path:
        try:
            # Load the image and display it in the GUI
            image = Image.open(file_path)
            image = image.resize((300, 300))
            image_tk = ImageTk.PhotoImage(image)
            canvas.itemconfig(canvas_image, image=image_tk)
            canvas.image = image_tk
            # Classify the image
            classify_image(file_path)
        except:
            # Show an error message if there was a problem loading the image
            messagebox.showerror('Error', 'Could not open the image file.')

# Create the GUI window
window = tk.Tk()
window.title('Image Classifier')

# Create a canvas to display the image
canvas = tk.Canvas(window, width=300, height=300)
canvas.pack()
canvas_image = canvas.create_image(0, 0, anchor=tk.NW)

# Create a button to upload the image
upload_button = tk.Button(window, text='Upload Image', command=handle_upload)
upload_button.pack()

# Start the GUI main loop
window.mainloop()


1/1 [==============================] - 0s 21ms/step
